In [4]:
import keras.models
import os
from scipy.io import wavfile
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from python_speech_features import mfcc
import pickle

class Config:
    def __init__(self, mode='conv', nfilt=26, nfeat = 13, nfft = 512, rate = 16000): #filtered out
        self.mode = mode
        self.nfilt = nfilt
        self.nfeat = nfeat
        self.rate = rate
        self.nfft = nfft
        self.step = int(rate/10) #0.1 sec, how much data computing while creating window
        self.model_path = os.path.join('/content/gdrive/My Drive/models', mode + '.model')
        self.p_path = os.path.join('/content/gdrive/My Drive/pickles', mode + '.p')


def build_predictions(audio_dir):
    y_true = []
    y_pred = []
    fn_prob = {}

    print('Extracting features from audio')
    for fn in tqdm(os.listdir(audio_dir)):
        print(audio_dir)
        print(fn)
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        label = fn2class[fn]
        c = classes.index(label)
        y_prob = []

        for i in range(0, wav.shape[0]-config.step, config.step):#cannot go further
            sample = wav[i:i+config.step]
            x = mfcc(sample, rate, numcep = config.nfeat, nfilt = config.nfilt, nfft = config.nfft)
            x = (x - config.min)/(config.max - config.min)#range
            if config.mode == 'conv':
                x = x.reshape(1, x.shape[0], x.shape[1], 1)
            elif config.mode=='time':
                x = np.expand_dims(x, axis = 0) #expand to 1 sample
            elif config.mode == 'convtime':
                x = x.reshape(1, x.shape[0], x.shape[1], 1)
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)

        fn_prob[fn] = np.mean(y_prob, axis = 0).flatten() #or that would be crap

    return y_true, y_pred, fn_prob

df = pd.read_csv('/content/gdrive/My Drive/valrandmus10.csv')#can remove if no classification
classes = list(np.unique(df.label))#all names of genres 

print(classes)

fn2class = dict(zip(df.fname, df.label))
p_path = os.path.join('/content/gdrive/My Drive/pickles', 'conv.p')

with open(p_path, 'rb') as handle:
    config = pickle.load(handle)

model = load_model("/content/gdrive/My Drive/models/conv.model")

y_true, y_pred, fn_prob = build_predictions('/content/gdrive/My Drive/valrandmus1/')
acc_score = accuracy_score(y_true = y_true, y_pred = y_pred)

y_probs = []
for i,row in df.iterrows():
    print(row.fname)
    y_prob = fn_prob[row.fname]
    y_probs.append(y_prob)
    for c ,p in zip(classes, y_prob):
        df.at[i, c] = p

print(classes)
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred

df.to_csv('/content/gdrive/My Drive/convconvconv.csv', index = False)




['Electronical', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']


  0%|          | 0/9 [00:00<?, ?it/s]

Extracting features from audio
/content/gdrive/My Drive/valrandmus1/
itstime.wav


 11%|█         | 1/9 [00:06<00:53,  6.65s/it]

/content/gdrive/My Drive/valrandmus1/
melqueen.wav


...

/content/gdrive/My Drive/valrandmus1/
deutsch.wav


 89%|████████▉ | 8/9 [00:31<00:03,  3.20s/it]

/content/gdrive/My Drive/valrandmus1/
ishtu.wav


100%|██████████| 9/9 [00:33<00:00,  2.88s/it]

battlecry.wav
blood.wav
deutsch.wav
ishtu.wav
itstime.wav
melqueen.wav
monster.wav
sonne.wav
warriors.wav
['Electronical', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']


In [0]:
!pip install python_speech_features

  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
